In [1]:
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from keras.applications import VGG16
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [2]:
pip install split-folders

Note: you may need to restart the kernel to use updated packages.


In [3]:
import splitfolders
import os
path = "/kaggle/input/hydrates-dataset/Hydrates dataset"
splitfolders.ratio(path,seed=1337, output="Hydrates_split", ratio=(0.6, 0.2, 0.2))

Copying files: 1169 files [00:07, 155.29 files/s]


In [4]:

train = '/kaggle/working/Hydrates_split/train'
val = '/kaggle/working/Hydrates_split/val'
test = '/kaggle/working/Hydrates_split/test'

# The shape of the RGB image
img_width, img_height, channels = 150, 150, 3  # you can try different sizes

# input shape
input_shape = (img_width, img_height, 3)
# position matters!
# Number_of_channels can be at the first or the last position
# in our case - "channels last"

# minibatch size
batch_size = 64
# train set size
nb_train_samples = 700
# validation set size
nb_validation_samples = 200
# test set size
nb_test_samples = 200

In [5]:
datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = datagen.flow_from_directory(
    test,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

train_generator = datagen.flow_from_directory(
    train,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

val_generator = datagen.flow_from_directory(
    val,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 236 images belonging to 2 classes.
Found 700 images belonging to 2 classes.
Found 233 images belonging to 2 classes.


In [6]:
import tensorflow as tf

# Enable DirectML backend
tf.config.experimental.enable_tensor_float_32_execution(False)

In [7]:
from tensorflow.keras.applications import ResNet50V2

resnet_net = ResNet50V2(weights='imagenet',
                        include_top=False,
                        input_shape=(150, 150, 3))
resnet_net.trainable = False

resnet_net.summary()

94668760/94668760 [==============================] - 3s 0us/step
Model: "resnet50v2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 150, 150, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 156, 156, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 75, 75, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 pool1_pad (ZeroPadding2D)   (None, 77, 77, 64)           0         ['conv1_conv[0][0]']          
                        

In [8]:
# Create a Sequential model
model = Sequential()

# Add the pre-trained ResNet50V2 model
model.add(resnet_net)

# Flatten the output of ResNet50V2
model.add(Flatten())

# Add a Dense layer with 512 neurons, ReLU activation, and Dropout for regularization
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

# Add another Dense layer with 256 neurons, ReLU activation, and Dropout for regularization
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

# Add the final Dense layer with 1 neuron and sigmoid activation for binary classification
model.add(Dense(1, activation='sigmoid'))

# Display the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 5, 5, 2048)        23564800  
                                                                 
 flatten (Flatten)           (None, 51200)             0         
                                                                 
 dense (Dense)               (None, 512)               26214912  
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 2

In [9]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=1e-5),
              metrics=['accuracy'])

In [10]:
fairy_tale = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs= 25,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size,
    workers=8)

/tmp/ipykernel_20/2543730722.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  fairy_tale = model.fit_generator(


Epoch 1/25
10/10 [==============================] - 35s 3s/step - loss: 1.4941 - accuracy: 0.5645 - val_loss: 0.5108 - val_accuracy: 0.7656
Epoch 2/25
10/10 [==============================] - 25s 2s/step - loss: 0.7652 - accuracy: 0.7123 - val_loss: 0.3168 - val_accuracy: 0.8385
Epoch 3/25
10/10 [==============================] - 26s 2s/step - loss: 0.5750 - accuracy: 0.7814 - val_loss: 0.2449 - val_accuracy: 0.8750
Epoch 4/25
10/10 [==============================] - 25s 2s/step - loss: 0.4700 - accuracy: 0.8255 - val_loss: 0.1737 - val_accuracy: 0.9167
Epoch 5/25
10/10 [==============================] - 26s 3s/step - loss: 0.3816 - accuracy: 0.8344 - val_loss: 0.1743 - val_accuracy: 0.9219
Epoch 6/25
10/10 [==============================] - 30s 3s/step - loss: 0.3631 - accuracy: 0.8789 - val_loss: 0.1168 - val_accuracy: 0.9427
Epoch 7/25
10/10 [==============================] - 25s 2s/step - loss: 0.2602 - accuracy: 0.8915 - val_loss: 0.1101 - val_accuracy: 0.9531
Epoch 8/25
10/10 [==